In [5]:
import skimage as sk
import numpy as np
import os
import napari
import pandas as pd
from glob import glob
from tqdm import tqdm

In [6]:
def get_measurements(mask,props, voxels):
    filtered_masks = sk.segmentation.clear_border(mask)
    df = sk.measure.regionprops_table(filtered_masks,properties=props, spacing=voxels)
    df = pd.DataFrame.from_dict(df)
    return filtered_masks, df

In [12]:
def get_3D_surfacearea_and_sphericity(filtered_masks,df,voxels):
    surf_area_list = []
    sphericity_list = []
    vol = np.asarray(df['area']).astype(np.float64)
    c = 0
    for i in np.asarray(df['label']).astype(int):
        obj = filtered_masks == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0,spacing=(voxels[0],voxels[1],voxels[2]))
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        surf_area_list.append(surf_area)
        sphericity = (np.pi**(1/3)*((6*vol[c])**(2/3)))/surf_area
        sphericity_list.append(sphericity)
        c += 1
    surface_areas = pd.Series(surf_area_list,name='Surface_Area (um^2)')
    sphericities = pd.Series(sphericity_list,name='Sphericity')
    merged_df = pd.concat([df,surface_areas,sphericities], axis=1)
    return merged_df
    

In [8]:
def save(save_path, img_name, filtered_masks, merged_df):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        dataframe_path = os.path.join(save_path,'measurements')
    sk.io.imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'.tif'),filtered_masks,check_contrast=False)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

In [4]:
files = sorted(glob('E:/Lempradl_Lab/April/OIC-78_Drosophila-brain-lesion-quant/April_manual_annotation/*.tif'))
labels = list(map(sk.io.imread,files))

In [10]:
viewer = napari.view_labels(labels[10])

In [15]:
props = ('label','area')
voxels = [3.0,0.208,0.208]
save_path = 'E:/Lempradl_Lab/April/OIC-78_Drosophila-brain-lesion-quant/April_manual_annotation/'

for i in tqdm(range(len(labels))):
    img_name = os.path.basename(files[i])
    mask_img, df = get_measurements(mask=labels[i],props=props,voxels=voxels)
    merged_df = get_3D_surfacearea_and_sphericity(filtered_masks=mask_img,df=df,voxels=voxels)
    save(save_path=save_path, img_name=img_name, filtered_masks=mask_img, merged_df=merged_df)

100%|██████████| 58/58 [2:18:16<00:00, 143.04s/it]  
